<a href="https://colab.research.google.com/github/Kimyobu/Stable-diffusion-a1111-colab/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title # <font size='7'> <font color="#3d73f2">**Comfy</font><font color="#3db6f2">U</font>I** <font size="6"> <font color="#ff8f4a">Environment Setup <font color="white">⬇️/🔃
%cd /content
from pathlib import Path
from google.colab import drive,output
from IPython.display import display, Markdown, Latex
import os
def install_req():
    %cd $WORKSPACE/custom_nodes
    try:
        for node in os.listdir():
            reqfile = f"./{node}/requirements.txt"
            if os.path.isfile(reqfile):
                !pip install -r {reqfile}
        output.clear()
    except Exception as Error:
        print(Error)
    print('Install Requirement Success')
    %cd $WORKSPACE

def MD(markdownstring):
    return display(Markdown(markdownstring))

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    if not os.path.isdir('/content/drive'):
        !echo "Mounting Google Drive..."
        drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117
%cd ./custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd $WORKSPACE
output.clear()
MD('<font color="lime"> SetUP Envoriment Complete✓')

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

# <font size="6" color="#ff4aff">**Models**<font color="cyan"> Manager🌀

In [ ]:
#@title <font color="cyan">Downloader⬇️
#@markdown ##Input Your Download URL and Model Directory

#@markdown See <a href='#scrollTo=brFe7v0dAY48'>Docs</a>.
import os.path as Path
WORKSPACE
%cd $WORKSPACE
model_url = 'https://civitai.com/api/download/models/104291' #@param {type:"string"}
model_dir = './models/loras' #@param {type:"string"}
model_name = 'hanfu-tang_lora' #@param {type:"string"}
model_type = "safetensors" # @param ["safetensors", "pth"] {allow-input: true}
save_name = f'{model_name}.{model_type}'
args = f'-O {Path.join(model_dir,save_name)}' if model_name != '' else f'-P {model_dir}'
!mkdir -p {model_dir}
!wget -c {args} {model_url}

In [ ]:
#@title <font color="orange">**Goo<font color="#20b000">gle_<font color="#70bcff">Drive** <font color="cyan">Downloader⬇️
#@markdown ##Input Your Download URL and Save Directory

#@markdown See <a href='#scrollTo=brFe7v0dAY48'>Docs</a>.

#@markdown * Save Model To Drive
import os.path as Path
import os
import json
from google.colab import drive,output
WORKSPACE
%cd /content
if not os.path.isdir('/content/drive'):
        !echo "Mounting Google Drive..."
        drive.mount('/content/drive')
%cd $WORKSPACE
SAVE_dir = '/content/drive/MyDrive/Save_Data' #@param {type:"string"}
SAVE_As = 'models.json' #@param ["models.json"]
check_list = False #@param {type:"boolean"}
uninstall_model = False #@param {type:"boolean"}
uninstall_dir = False #@param {type:"boolean"}
import_to_workspcae = True #@param {type:"boolean"}
model_url = 'https://civitai.com/api/download/models/104291' #@param {type:"string"}
model_dir = './models/loras' #@param {type:"string"}
model_name = 'hanfu-tang_lora' #@param {type:"string"}
model_type = "safetensors" # @param ["safetensors", "pth"] {allow-input: true}
save_name = f'{model_name}.{model_type}'
args = f'-O {Path.join(model_dir,save_name)}' if model_name != '' else f'-P {model_dir}'

!mkdir -p {SAVE_dir}
JSON = Path.join(SAVE_dir,SAVE_As)

if not Path.isfile(JSON):
    !echo -= Creating DataBase File =-
    !echo '{}' > $JSON

r = open(JSON,'r')
Models = json.loads(r.read())
r.close()

def check():
    print(f'\nCurrent List:\n{json.dumps(Models,indent=2)}\n')

def update():
    r = open(JSON,'w')
    r.write(json.dumps(Models,indent=2))
    r.close()

print('\n')
msg = save_name if model_name != '' else model_url
if check_list:
    output.clear()
    check()
elif uninstall_dir:
    if model_dir in Models:
        Models.pop(model_dir)
        print(f'Remove Directory {model_dir}')
        check()
elif uninstall_model:
    if model_dir in Models:
        m = Models[model_dir]
        target = [x for x in m if x['url']==model_url or x['name']==model_name]
        if len(target) != 0:
            for x in target:
                m.remove(x)
                n = x['name']
                u = x['url']
                print(f'Remove {n}/{u} in {model_dir}')
            print(f'Remove All match {msg}/{model_url} from {model_dir}')
        else:
            print(f'No match {msg}/{model_url} in List')
        Models[model_dir] = m
        if len(m) == 0:
            Models.pop(model_dir)
            print(f'Remove Empty Directory {model_dir}')
        check()
    else:
        print(f'{model_dir} is Empty')
else:
    if not isinstance(Models.get(model_dir),list):
        Models.setdefault(model_dir,[])
    data = {'url':model_url,'name':model_name,'type':model_type}
    if data in Models[model_dir]:
        print('\nThe file is already fully retrieved; nothing to do.\n')
    else:
        Models[model_dir].append(data)
        print(f'\nAdd {msg} in {model_dir}\n')
    if import_to_workspcae:
        print('Wait for Import File to Workspcae\n')
        !wget -c {model_url} {args}

update()

In [ ]:
#@title Import All Saved Model From Google_Drive⬇️💾
import os
import os.path as Path
from google.colab import drive
WORKSPACE
%cd /content
if not os.path.isdir('/content/drive'):
        !echo "Mounting Google Drive..."
        drive.mount('/content/drive')
%cd $WORKSPACE
SAVE_dir = '/content/drive/MyDrive/Save_Data' #@param {type:"string"}
OPEN_As = 'models.json' #@param ["models.json"]
File_Path = Path.join(SAVE_dir,OPEN_As)
if not Path.isdir(SAVE_dir):
    print(f'Path: {SAVE_dir} is not directory!!')
elif not Path.isfile(File_Path):
    print(f'File: {OPEN_As} not found ({File_Path})')

r = open(JSON,'r')
Models = json.loads(r.read())
r.close()

def check():
    print(f'\nCurrent List:\n{json.dumps(Models,indent=2)}\n')

check()
for x in Models:
    !echo -= Import $x =-
    model_dir = x
    for m in Models[x]:
        model_name = m['name']
        model_type = m['type']
        model_url = m['url']
        save_name = f'{model_name}.{model_type}'
        args = f'-O {Path.join(model_dir,save_name)}' if model_name != '' else f'-P {model_dir}'
        !mkdir -p {model_dir}
        !wget -c {model_url} {args}

In [ ]:
#@markdown Recommander Checkpoints By ComfyUI⬇

### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/


# SD1.5
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
#!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
#!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
#!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora


# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
#!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin


# ControlNet
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/


# Controlnet Preprocessor nodes by Fannovel16
#!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
#!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
#!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
#!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
#!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/

#**<font color="white" size="6">Custom Node ⚙️**

In [ ]:
#@title <font color="yellow" size="5"> Install Custom Node⬇️
WORKSPACE
install_all_node_requirements = False #@param {type: "boolean"}
clone_node_project = 'https://github.com/gamert/ComfyUI_tagger.git' #@param {type:"string"}
%cd $WORKSPACE/custom_nodes
!git clone {clone_node_project}
import os.path as Path
repo = f'./{Path.basename(os.path.splitext(clone_node_project)[0])}'
reqfile = f'{repo}/requirements.txt'
if Path.isfile(reqfile):
    !pip install -r {reqfile}
else:
    print('Requirements file not found')
if install_all_node_requirements:
    install_req()

In [ ]:
#@title <font color="lime" size="5"> Update Custome Node 🔃
import os
WORKSPACE
p = f'{WORKSPACE}/custom_nodes'
%cd $p
Update_All_Nodes = True #@param {type: "boolean"}
Force_Pull = False #@param {type: "boolean"}
#@markdown <font color="red" size="4">Force Pull Will Overwrite all file !!<br>Force Pull จะเขียนทับไฟล์ทั้งหมด !!</font>
Node_folder = './ComfyUI-Manager' #@param {type: "string"}
if Update_All_Nodes:
    for x in os.listdir():
        x = os.path.join(p,x)
        if os.path.isdir(x):
            %cd {x}
            !git pull
else :
    %cd $Node_folder
    !git fetch --all
    !git reset --hard origin
    !git pull

#**<font size="7" color="orange">Launch <font color="#3d73f2">Web<font color="#3db6f2">U<font color="white">I 🚀**

In [ ]:
#@title <font size="5" color="lime">Run <font color="#3d73f2">**Comfy</font><font color="#3db6f2">U<font color="white">I** <font color="pink">with <font color="orange">cloudflared
# @markdown ### Run ComfyUI with cloudflared (Recommended Way)<br>รัน ComfyUI ด้วย cloudflared (วิธีปกติ)
WORKSPACE
%cd /content
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
install_all_node_requirements = True #@param {type: "boolean"}
if install_all_node_requirements:
    install_req()
%cd $WORKSPACE
from google.colab import output
import subprocess
import threading
import time
import socket
import urllib.request
output.clear()

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--loglevel", "info", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')

args = '--preview-method auto --dont-print-server --highvram' #@param {type: "string"}
#@markdown See <a href='#scrollTo=brFe7v0dAY48'>Docs</a>.
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
!python main.py {args}

In [ ]:
#@title <font size="5" color="lime">Run <font color="#3d73f2">**Comfy</font><font color="#3db6f2">U<font color="white">I** <font color="pink">with <font color="red">Local<font color="white">tunnel
# @markdown ### Run ComfyUI with localtunnel (Optional)<br>รัน ComfyUI ด้วย localtunnel (แล้วแต่เลือกใช้)
WORKSPACE
%cd $WORKSPACE
install_all_node_requirements = True #@param {type: "boolean"}
if install_all_node_requirements:
    install_req()
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
args = '--preview-method auto --dont-print-server --highvram' #@param {type: "string"}
#@markdown See <a href='#scrollTo=brFe7v0dAY48'>Docs</a>.
!python main.py {args}

In [ ]:
# @title ##<font color="yellow">File Explorer📁
!pip install -U imjoy-elfinder
import threading
from google.colab import output
from imjoy_elfinder.app import main
output.clear()

PORT = 4321 #@param {type:"number"}
root_dir = "/content"
open_in_new_tab = False

def start_file_explorer(root_dir=root_dir, port=PORT):
    try:
        main(["--root-dir=" + root_dir, "--port=" + str(port)])
    except Exception as e:
        print("Error starting file explorer:", str(e))


def open_file_explorer(open_in_new_tab=False, root_dir=root_dir, port=PORT):
    thread = threading.Thread(target=start_file_explorer, args=[root_dir, port])
    thread.start()
    output.serve_kernel_port_as_iframe(port, height="500")

open_file_explorer(port=PORT)

---
#Docs
Args | Value
       --- | ---
            URL | URL to file or URL to download
            Dir | Path of Directory to save file
            Name | Name of file to save (rename file)
            Type | Type of model (extension of file)

---

###Downloader:
   Args | Value
           --- | ---
                URL | https://civitai.com/api/download/models/138079
                Dir | ./models/checkpoints
                Name | ReproductionAnimeSDXL
                Type | safetensors

   ```
  Length: 6938042036 (6.5G)
  Saving to: ‘$WORKSPACE/models/checkpoints/ReproductionAnimeSDXL.safetensors’

  models/Reproductio... 100%[===================>]   6.46G  58.4MB/s    in 2m 13s

  2023-08-13 03:41:16 (49.6 MB/s) - ‘$WORKSPACE/models/checkpoints/ReproductionAnimeSDXL.safetensors’ saved [6938042036/6938042036]
   ```
   Args | Value
           --- | ---
                URL | https://civitai.com/api/download/models/138079
                Dir | /content/drive/MyDrive/MyModels/checkpoints
                Name | GuoFeng4XL
                Type | safetensors

   ```
  Length: 6938041020 (6.5G)
Saving to: ‘/content/drive/MyDrive/MyModels/checkpoints/GuoFeng4XL.safetensors’

/content/drive/MyDr 100%[===================>]   6.46G  14.0MB/s    in 10m 7s  

2023-08-13 04:07:32 (10.9 MB/s) - ‘/content/drive/MyDrive/MyModels/checkpoints/GuoFeng4XL.safetensors’ saved [6938041020/6938041020]
   ```
---

###CommandLine Argument:
Option | Description
---|---
        -h, --help | show this help message and exit
--listen [IP] | Specify the IP address to listen on (default: 127.0.0.1). If --listen is provided without an argument, it defaults to 0.0.0.0. (listens on all)
--port [PORT] | Set the listen port.
--enable-cors-header [ORIGIN] | Enable CORS (Cross-Origin Resource Sharing) with optional origin or allow all with default '*'.
--extra-model-paths-config [PATH (PATH . . .) ] | Load one or more extra_model_paths.yaml files.
--output-directory [OUTPUT_DIRECTORY] | Set the ComfyUI output directory.
--auto-launch | Automatically launch ComfyUI in the default browser.
--cuda-device [DEVICE_ID] | Set the id of the cuda device this instance will use.
--cuda-malloc | Enable cudaMallocAsync (enabled by default for torch 2.0 and up).
--disable-cuda-malloc | Disable cudaMallocAsync.
--dont-upcast-attention | Disable upcasting of attention. Can boost speed but increase the chances of black images.
--force-fp32 | Force fp32 (If this makes your GPU work better please report it).
--force-fp16 | Force fp16.
--fp16-vae | Run the VAE in fp16, might cause black images.
--bf16-vae | Run the VAE in bf16, might lower quality.
--directml [DIRECTML_DEVICE] | Use torch-directml.
--preview-method [none,auto,latent2rgb,taesd] | Default preview method for sampler nodes.
--use-split-cross-attention | Use the split cross attention optimization. Ignored when xformers is used.
--use-quad-cross-attention | Use the sub-quadratic cross attention optimization . Ignored when xformers is used.
--use-pytorch-cross-attention | Use the new pytorch 2.0 cross attention function.
--disable-xformers | Disable xformers.
--gpu-only | Store and run everything (text encoders/CLIP models, etc... on the GPU).
--highvram | By default models will be unloaded to CPU memory after being used. This option keeps them in GPU memory.
--normalvram | Used to force normal vram use if lowvram gets automatically enabled.
--lowvram | Split the unet in parts to use less vram.
--novram | When lowvram isn't enough.
--cpu | To use the CPU for everything (slow).
--dont-print-server | Don't print server output.
--quick-test-for-ci | Quick test for CI.
--windows-standalone-build | Windows standalone build: Enable convenient things that most people using the standalone windows build will probably enjoy (like auto opening the page on startup).
--disable-metadata | Disable saving prompt metadata in files.

---